# Recommender System Notebook
Deze notebook bouwt een content-based recommender voor de VKM-module dataset.

In [109]:
!pip install -r requirements.txt

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [110]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer


nltk.download("stopwords")
nltk.download("wordnet")

# Stopwoorden NL + ENG
stop_words = set(stopwords.words("english")) | set(stopwords.words("dutch"))

# Lemmatizer voor Engels
lemmatizer_en = WordNetLemmatizer()

# Stemmer voor Nederlands
stemmer_nl = SnowballStemmer("dutch")

df = pd.read_csv("Uitgebreide_VKM_dataset_zonder_weird_data.csv")
df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Storm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Storm\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,id,name,shortdescription,description,content,studycredit,location,contact_id,level,learningoutcomes,module_tags,interests_match_score,popularity_score,estimated_difficulty,available_spots,start_date
0,159,kennismaking met psychologie,"brein, gedragsbeinvloeding, ontwikkelingspsych...",in deze module leer je hoe je gedrag van jezel...,in deze module leer je hoe je gedrag van jezel...,15,den bosch,58,nlqf5,a. je beantwoordt vragen in een meerkeuze kenn...,"['brein', 'gedragsbeinvloeding', 'ontwikkeling...",0.54,319,1,79,2025-12-24
1,160,learning and working abroad,"internationaal, persoonlijke ontwikkeling, ver...",studenten kiezen binnen de (stam) van de oplei...,studenten kiezen binnen de (stam) van de oplei...,15,den bosch,58,nlqf5,de student toont professioneel gedrag conform ...,"['internationaal', 'persoonlijke', 'ontwikkeli...",0.92,172,5,56,2025-12-20
2,161,proactieve zorgplanning,"proactieve zorgplanning, cocreatie, ziekenhuis",het jeroen bosch ziekenhuis wil graag samen me...,het jeroen bosch ziekenhuis wil graag samen me...,15,den bosch,59,nlqf5,de student past pro actieve zorgplanning toe b...,"['proactieve', 'zorgplanning', 'cocreatie', 'z...",0.78,217,5,55,2025-09-23
3,162,rouw en verlies,"rouw & verlies, palliatieve zorg & redeneren, ...",in deze module wordt stil gestaan bij rouw en ...,in deze module wordt stil gestaan bij rouw en ...,30,den bosch,58,nlqf6,de student regisseert en voert (deels) zelfsta...,"['rouw', 'verlies', 'palliatieve', 'zorg', 're...",0.69,454,1,54,2025-10-25
4,163,acuut complexe zorg,"acute zorg, complexiteit, ziekenhuis, revalidatie",in deze module kunnen studenten zich verdiepen...,in deze module kunnen studenten zich verdiepen...,30,den bosch,58,nlqf6,de student regisseert en voert (deels) zelfsta...,"['acute', 'zorg', 'complexiteit', 'ziekenhuis'...",0.40,178,5,38,2025-11-19


## Data schoonmaken

In [111]:

def detect_language(text):
    # Mini-detectie op basis van veelgebruikte woorden
    dutch_keywords = ["de", "het", "een", "en", "je", "jij", "wij", "zijn", "module", "leren", "opleiding"]
    english_keywords = ["the", "a", "an", "and", "is", "are", "course", "learn"]

    text_low = text.lower()
    nl_score = sum(1 for w in dutch_keywords if w in text_low)
    en_score = sum(1 for w in english_keywords if w in text_low)

    return "nl" if nl_score >= en_score else "en"

def clean_text_nlp(text):

    if not isinstance(text, str) or text.strip() == "" or text.lower() in ["ntb", "tbd", "nader te bepalen"]:
        return "ntb"
    
    # Lowercase
    text = text.lower()

    # Verwijder cijfers en speciale tekens
    text = re.sub(r"[^a-zA-Záéíóúàèçäëïöüñ\s]", " ", text)

    # Normaliseer spaties
    text = re.sub(r"\s+", " ", text).strip()

    # Taal bepalen
    lang = detect_language(text)

    # Stopwoorden
    words = [w for w in text.split() if w not in stop_words]

    # NL = stemmer  
    if lang == "nl":
        words = [stemmer_nl.stem(w) for w in words]

    # EN = lemmatizer  
    else:
        words = [lemmatizer_en.lemmatize(w) for w in words]

    return " ".join(words) if words else "ntb"


# ===============================
# APPLY TO SHORTDESCRIPTION
# ===============================

df["shortdescription"] = df["shortdescription"].apply(clean_text_nlp)

df[["shortdescription"]].head()


for col in df.columns:
    if df[col].dtype == object:
        df[col] = df[col].str.lower()

df["combined_text"] = (
    df["name"].astype(str) + " " +
    df["shortdescription"].astype(str) + " " +
    df["module_tags"].astype(str) + " " +
    df["location"].astype(str)
)
df.head()

,id,name,shortdescription,description,content,studycredit,location,contact_id,level,learningoutcomes,module_tags,interests_match_score,popularity_score,estimated_difficulty,available_spots,start_date,combined_text
0,159,kennismaking met psychologie,brein gedragsbeinvloed ontwikkelingspsychologi...,in deze module leer je hoe je gedrag van jezel...,in deze module leer je hoe je gedrag van jezel...,15,den bosch,58,nlqf5,a. je beantwoordt vragen in een meerkeuze kenn...,"['brein', 'gedragsbeinvloeding', 'ontwikkeling...",0.54,319,1,79,2025-12-24,kennismaking met psychologie brein gedragsbein...
1,160,learning and working abroad,international person ontwikkel verpleegkund,studenten kiezen binnen de (stam) van de oplei...,studenten kiezen binnen de (stam) van de oplei...,15,den bosch,58,nlqf5,de student toont professioneel gedrag conform ...,"['internationaal', 'persoonlijke', 'ontwikkeli...",0.92,172,5,56,2025-12-20,learning and working abroad international pers...
2,161,proactieve zorgplanning,proactieve zorgplanning cocreatie ziekenhuis,het jeroen bosch ziekenhuis wil graag samen me...,het jeroen bosch ziekenhuis wil graag samen me...,15,den bosch,59,nlqf5,de student past pro actieve zorgplanning toe b...,"['proactieve', 'zorgplanning', 'cocreatie', 'z...",0.78,217,5,55,2025-09-23,proactieve zorgplanning proactieve zorgplannin...
3,162,rouw en verlies,rouw verlies palliatiev zorg redener trauma cu...,in deze module wordt stil gestaan bij rouw en ...,in deze module wordt stil gestaan bij rouw en ...,30,den bosch,58,nlqf6,de student regisseert en voert (deels) zelfsta...,"['rouw', 'verlies', 'palliatieve', 'zorg', 're...",0.69,454,1,54,2025-10-25,rouw en verlies rouw verlies palliatiev zorg r...
4,163,acuut complexe zorg,acute zorg complexiteit ziekenhuis revalidatie,in deze module kunnen studenten zich verdiepen...,in deze module kunnen studenten zich verdiepen...,30,den bosch,58,nlqf6,de student regisseert en voert (deels) zelfsta...,"['acute', 'zorg', 'complexiteit', 'ziekenhuis'...",0.40,178,5,38,2025-11-19,acuut complexe zorg acute zorg complexiteit zi...


## Vectorisatie

In [112]:
stopwords_nl = stopwords.words("dutch")
stopwords_en = stopwords.words("english")

combined_stopwords = set(stopwords_nl + stopwords_en)

vectorizer = CountVectorizer(stop_words=list(combined_stopwords))
vectorized = vectorizer.fit_transform(df["combined_text"])
vectorized

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 3095 stored elements and shape (211, 1615)>

## Cosine Similarity

In [113]:
similarities = cosine_similarity(vectorized)
similarity_df = pd.DataFrame(similarities, index=df["name"], columns=df["name"])
similarity_df.head()

name,kennismaking met psychologie,learning and working abroad,proactieve zorgplanning,rouw en verlies,acuut complexe zorg,kraam kind en jeugd,"profiel specifieke modulen welzijn & samenleving, jeugd, zorg","happy city, happy people design lab",building & sustaining vital communities,ggz agoog,...,tekenen,performance in art,material ecologies,art & humanity (nieuwe naam formuleren),research in art istic practice,ethiek & kritisch denken,avans innovative studio junior,avans innovative studio senior,succesvol managen van een project,de stem van je geweten. ga opzoek naar jouw moreel kompas.
name,,,,,,,,,,,,,,,,,,,,,
kennismaking met psychologie,1.000000,0.130744,0.089087,0.076472,0.087538,0.108148,0.092450,0.056344,0.076472,0.078567,...,0.084667,0.0,0.0,0.098295,0.069505,0.062439,0.00000,0.0,0.0,0.041667
learning and working abroad,0.130744,1.000000,0.104828,0.089984,0.103005,0.127257,0.108786,0.066299,0.089984,0.092450,...,0.099627,0.0,0.0,0.115663,0.081786,0.073472,0.20601,0.0,0.0,0.000000
proactieve zorgplanning,0.089087,0.104828,1.000000,0.061314,0.210559,0.086711,0.074125,0.045175,0.061314,0.062994,...,0.067884,0.0,0.0,0.078811,0.055728,0.050063,0.00000,0.0,0.0,0.000000
rouw en verlies,0.076472,0.089984,0.061314,1.000000,0.240990,0.074432,0.127257,0.038778,0.052632,0.054074,...,0.058272,0.0,0.0,0.067651,0.047836,0.042974,0.00000,0.0,0.0,0.000000
acuut complexe zorg,0.087538,0.103005,0.210559,0.240990,1.000000,0.085203,0.182089,0.044390,0.060248,0.061898,...,0.066704,0.0,0.0,0.077440,0.054759,0.049192,0.00000,0.0,0.0,0.000000


## Aanbevelingsfunctie

In [ ]:
def recommend(module_name, similarity_df):
    try:
        recs = similarity_df[module_name].sort_values(ascending=False)[1:6]
        print(f"Aanbevolen modules voor '{module_name}':")
        print(recs)
    except KeyError:
        print("Module niet gevonden in dataset.")
        print("Beschikbare modules zijn:")
        print(similarity_df.index.tolist())

recommend("kennismaking met psychologie", similarity_df)

Aanbevolen modules voor 'kennismaking met psychologie':
name
minor forensisch onderzoek in de rechtbank- (if/ka)    0.166667
business innovation                                    0.136083
learning and working abroad                            0.130744
duurzaamheid/ csrd/ circulariteit,                     0.130744
tutorial club                                          0.130744
Name: kennismaking met psychologie, dtype: float64
